# Markov Chain Stationary Distribution

### What is a Markov Chain?

A Markov Chain is a stochastic process where the probabilty of transitoning from one state to another only depends on the previous state. In other words, it is a sequence of random variable $X_0$, $X_1$, ..., $X_n$, ... where the information of $X_n$ is solely related with $X_{n-1}$, which is referred to as Markov Property.
The transition probabilities in a Markov process can be described as a transition matrix such that $p_{ij}$ is the transition probability from state i to state j; $p_{ij} = P(X_{n+1} = j | X_n = i)$.

$$P = \begin{bmatrix}1/3 & 1/3 & 1/3 & 0\\
0 & 0 & 1/2 & 1/2 \\
0 & 1 & 0 & 0 \\
1/2 & 0 & 0 & 1/2
\end{bmatrix}$$

For a small transition matrix, it gets easier to represent it as a transition diagram.

<p align="center">
    <img width="300" height="250" src="../imgs/markov_chain_diagram_img.001.png">
</p>

The n-step transition probability is calculated as $P^n$. For example, the probability that the chain is in state 3 after 5 steps, starting at state 1 is the (1,3) entry of $P^5$, in other words, $P(X_5 = 3 | X_0 = 3)$ is approximately equal to 0.32

In [91]:
import numpy as np
import numpy.linalg as LA

P = np.array(
    [[1/3, 1/3, 1/3, 0],
     [0, 0, 1/2, 1/2],
     [0, 1, 0, 0],
     [1/2, 0, 0, 1/2]]
)

P_5 = LA.matrix_power(P, 5)
P_5[1,3]

0.3159722222222222

Let $t = (t_1, t_2, ..., t_M)$ the initial condition of the chain where $M$ is the number of the states. The vector $t$ is the PMF of $X_0$ from which $X_0$ is randomly chosen. Then, the marginal distribution of the chain at step $n$ can be computed as $tP^n$. For example, given a initial condition of $t = (1/4, 1/4, 1/4, 1/4)$, the marginal probability of the chain being at state 3 is the third element of $tP^5$. It is approximately equal to 0.2, meaning that the chain at step 5 will be in state 3 with the probabilty of 1/5, given the equal probability of starting in each of the four states. 

In [44]:
t = np.array([1/4, 1/4, 1/4, 1/4])
tP_5 = t@P_5
tP_5[2]

0.20781893004115226

### Stationary Distribution

When the number of steps get larger and larger and goes to the infinity, the long-run behavior of the chain is described by the stationary distribution or the steady-state distribution. Intuitively, the stationary distribution is the proportion of the time the chain spends at each state in a long run. For example, the following stationary distribution $s$ means that the chain spends 2/3 of its time at state 1 in a long run. 

$$s = \begin{bmatrix} 2/3 & 1/2 \end{bmatrix}$$

We can check the stationary distribution of $P$ by multiplying it 1000 times. One of the interesting parts of $P^{1000}$ is that its rows are all identical. We will see the reason behind it later.

In [45]:
P_1000 = LA.matrix_power(P, 1000)
P_1000

array([[0.21428571, 0.28571429, 0.21428571, 0.28571429],
       [0.21428571, 0.28571429, 0.21428571, 0.28571429],
       [0.21428571, 0.28571429, 0.21428571, 0.28571429],
       [0.21428571, 0.28571429, 0.21428571, 0.28571429]])

The stationary distribution $s$ for a Markov chain with transition matrix $P$ is a row vector $s = (s_1, ..., s_M)$ such that $s_i >= 0$ and $\sum_{i}^{}s_i = 1$ with the following condition.

$$sP = s$$

The above condition means that $s$ is a left eigenvector of $P$ with eigenvalue 1. 

Each row of the transition matrix $P^n$ converges to $s$ as $n$ goes to the infinity, meaning that in a long run, $P(X_n = i) = s_i$.
Using eigenvalue decomposition, we can see this is the case. 

By eigenvalue decomposition, we get the follwing. 
$$ P = Q{\Lambda}Q^{-1} $$
, where $Q$ is the square M x M matrix whose ith column is the eigenvector $q_i$ of $P$ and ${\Lambda}$ is the diagonal matrix whose diagonal elements are the corresponding eigenvalues. By taking the inverse of $Q$ on both sides, we get the following.

$$ Q^{-1}P = {\Lambda}Q^{-1} $$

Now, the rows of $Q^{-1}$ are the left eigenvectors of $P$. The row of $Q^{-1}$ that corresponds to the eigenvalue 1 in ${\Lambda}$ is the last row. As we can see, the last row of $Q^{-1}$ after normalization matches with $s$ as expected. 

As per normalization, $numpy.linalg.eig(P)$ has no reason to return a eigenvector that is a PMF. Therefore, the forth row should be normalized to become a probability distribution. 

In [90]:
lam, Q = LA.eig(P)
inv_Q = LA.inv(Q)
qs = inv_Q.real[3,:] / np.sum(inv_Q.real[3,:])

print (lam.real)
print (inv_Q.real)
print (qs)


[-0.71984207  0.2765877   0.2765877   1.        ]
[[-0.15650063 -0.80423025  0.63108565  0.32964523]
 [-0.16987783 -0.21231393 -0.18864282  0.57083458]
 [-0.16987783 -0.21231393 -0.18864282  0.57083458]
 [-0.42857143 -0.57142857 -0.42857143 -0.57142857]]
[0.21428571 0.28571429 0.21428571 0.28571429]


The next question should be about the existence and uniqueness of the stationary distribution. The following conditition holds for any Markov Chain. 
* Any irreducible Markove chain has the unique stationary distribution, and in this distribution, every state has positive probabillity.

If we run a markov chain for a long time, does it converge to $s$? In other words, $P(X_n = i)$ converges to $s$ when $n$ goes to the infinity?. The following three conditions are all equivalent. 
* $P(X_n = i)$ converges to $s_i$ when $n$ goes to the infinity.
* There exists $m > 0$ such that the elements of $Q^m$ are all positive. 
* The chain is irreducible and aperiodic. 

The proof of this claim is beyond the scope of this article. 

### Initial condition and stationary distribution

One of the interesting parts about the stationary distribution is that it doesn't depend on the initial condition. In other words, regardless what the initial condition is, it all converges to the stationary distribution. Why is this the case? It takes several steps to explain this.


Let's consider a transition matrix P. One property of the matrix is that each row adds up to 1 because each one is a condtional PMF. Hence, one of its eigenvector must be a unit vector whose components are all one, that is **u** = (1, 1, 1, 1, 1 ...), and the corresponding eigenvalue is one as in the following.

$$P = \begin{bmatrix}1/3 & 1/3 & 1/3 & 0\\
0 & 0 & 1/2 & 1/2 \\
0 & 1 & 0 & 0 \\
1/2 & 0 & 0 & 1/2
\end{bmatrix}
\begin{bmatrix}1\\
               1\\
               1 \\
               1
\end{bmatrix} = 
1*
\begin{bmatrix}1\\
               1\\
               1 \\
               1
\end{bmatrix}
$$

Therefore, when we factroize P using eigenvalue decomposition, the column of Q that is a eigenvector of ${\lambda} = 1$ is **u**, and its components are all one. Also, as you see above, the left eigenvector of ${\lambda} = 1$ is the stationary distrubtion after normalization.

$$ P = Q{\Lambda}Q^{-1} $$

$$ s = \begin{bmatrix} 0.21428571 & 0.28571429 & 0.21428571 & 0.28571429 \end{bmatrix}$$

The column vector $u$ is multiplied with the row vector $s^T$ and they produce the long term behavor of the transition matrix. 


#### Powers of matrix
The powers of matrix P can be shown as below, and $v_i$ are the row vectors of $Q^{-1}$.

$$ P^n = Q{\Lambda}^nQ^{-1} = q_1{\lambda}_1^nv_1^T + q_2{\lambda}_2^nv_2^T + ... + q_4{\lambda}_4^nv_4^T  $$ 

When $n$ goes to the infinity, any term with an eigenvalue less than 1 will disappear. It can be shown that ${\lambda} = 1$ is the largest eigenvalue and it is guaranteed that any eigvalue cannot be greater than 1 by *Parron-Frobenius theorem*. With loss of generality, suppose ${\lambda}_1 = 1$, then any other terms will be zero as $n$ goes to the infinity.

Therefore we end up with the following. When $n$ goes to the infinity, 

$$ P^{\infty} = q_1{\lambda}_1^{\infty}v_1^T, \quad where \quad {\lambda}_1 = 1$$

$$ P^{\infty} = \begin{bmatrix}1\\
               1\\
               1 \\
               1
        \end{bmatrix} 
        \begin{bmatrix} 0.21428571 & 0.28571429 & 0.21428571 & 0.28571429 \end{bmatrix}
        $$

$P^{\infty}$ has all its rows identicial. For any initial condtition $t$ of the chain, $tP^{\infty}$ converges to the same stationary distribution because the sum of the components of t is 1. 

$$ s = \begin{bmatrix} 0.21428571 & 0.28571429 & 0.21428571 & 0.28571429 \\
0.21428571 & 0.28571429 & 0.21428571 & 0.28571429 \\
0.21428571 & 0.28571429 & 0.21428571 & 0.28571429 \\
0.21428571 & 0.28571429 & 0.21428571 & 0.28571429 \\
 \end{bmatrix}$$

Let's see it is true.

$$ Let \quad t = (t_1, t_2, ..., t_M),\quad where \quad \sum_{i}^{M}t_i = 1 \\

Therefore,\quad tP^{\infty} = t $$


Hopefully, you enjoy this notebook. If you have any question, feel free to ask me on the forum. Next time, I will cover some introductory topics over MCMC or VI. Thanks for reading this. 

### Reference

Introduction to Probability, ch11, Joseph K. Blitzstein, Jessica Hwang